In [5]:
# import libraries we need
%pylab inline
import tensorflow as tf
from tensorflow import keras
import tensorflow_probability as tfp
tfd = tfp.distributions
import sys
sys.path.append('./../simulation/')
import Simulator
import tensorflow_model_optimization as tfmot
import tempfile

Populating the interactive namespace from numpy and matplotlib


In [73]:
# load the trained model
trained_model = keras.models.load_model('g2model.h5')

# get weights from the trained model
def setup_pretrained_weights():
    model= trained_model

    _, pretrained_weights = tempfile.mkstemp('.tf')

    model.save_weights(pretrained_weights)

    return pretrained_weights

pretrained_weights = setup_pretrained_weights()

In [123]:
# create a base model
def make_model(input_shape):
    # input shape should be (time signal, 1)
    input_layer = keras.layers.Input(input_shape)

    # 1st con1d layer
    #conv1 = keras.layers.Conv1D(filters=64, kernel_size=4, padding='valid', strides=1, activation='relu')(input_layer)
    
    # 2nd con1d layer
    #conv2 = keras.layers.Conv1D(filters=64, kernel_size=4, padding='valid', strides=1, activation='relu')(conv1)

    # 3rd con1d layer
    #conv3 = keras.layers.Conv1D(filters=64, kernel_size=4, padding='valid', strides=1, activation='relu')(conv2)
    
    # maxpooling layer
    #pool = keras.layers.MaxPool1D(pool_size=3, strides=1, padding='valid')(conv3) # keras.layers.GlobalAveragePooling1D()(conv3) #  # 
    
    # flatten layer
    flat = keras.layers.Flatten()(input_layer)
    
    # fully connected layer to output a binary vector
    dense1 = keras.layers.Dense(2, activation='relu')(flat)
    #dense2 = keras.layers.Dense(2, activation='relu')(dense1)

    return keras.models.Model(inputs=input_layer, outputs=dense1)

base_model = make_model(input_shape=(200,1))
quantize_model = tfmot.quantization.keras.quantize_model

In [127]:
# annotate that only the Dense layers should be quantized.
def apply_quantization(layer):
    if isinstance(layer, tf.keras.layers.Dense):
        return tfmot.quantization.keras.quantize_annotate_layer(layer)
    elif isinstance(layer, tf.keras.layers.Conv1D):
        return tfmot.quantization.keras.quantize_annotate_layer(layer)
    elif isinstance(layer, tf.keras.layers.MaxPool1D):
        return tfmot.quantization.keras.quantize_annotate_layer(layer)
    return layer

annotated_model = tf.keras.models.clone_model(
    model = base_model,
    clone_function= apply_quantization )

q_aware_model = quantize_model(annotated_model)

q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [128]:
q_aware_model.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 200, 1)]          0         
_________________________________________________________________
quantize_layer_10 (QuantizeL (None, 200, 1)            3         
_________________________________________________________________
quant_flatten_18 (QuantizeWr (None, 200)               1         
_________________________________________________________________
quant_dense_17 (QuantizeWrap (None, 2)                 407       
Total params: 411
Trainable params: 402
Non-trainable params: 9
_________________________________________________________________


In [129]:
# fine tuning the mdoel
xtrain_subset = x_train[0:100] # out of 60000
ytrain_subset = y_train[0:100]

q_aware_model.fit(xtrain_subset, ytrain_subset,
                  batch_size=5, epochs=10, validation_split=0.1)


Epoch 1/10
9/9 [==============================] - 0s 10ms/step - loss: 0.6931 - accuracy: 0.5398 - val_loss: 0.6931 - val_accuracy: 0.6000
Epoch 2/10
9/9 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5572 - val_loss: 0.6931 - val_accuracy: 0.6000
Epoch 3/10
9/9 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.6271 - val_loss: 0.6931 - val_accuracy: 0.6000
Epoch 4/10
9/9 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5693 - val_loss: 0.6931 - val_accuracy: 0.6000
Epoch 5/10
9/9 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5229 - val_loss: 0.6931 - val_accuracy: 0.6000
Epoch 6/10
9/9 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5832 - val_loss: 0.6931 - val_accuracy: 0.6000
Epoch 7/10
9/9 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5717 - val_loss: 0.6931 - val_accuracy: 0.6000
Epoch 8/10
9/9 [=================

In [130]:
# quantized the model

converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()


INFO:tensorflow:Assets written to: /tmp/tmpwixwfqmw/assets


INFO:tensorflow:Assets written to: /tmp/tmpwixwfqmw/assets


In [95]:
with open('qaware_dummy.tflite', 'wb') as f:
    f.write(quantized_tflite_model)

The above method **only works for supported layers contained model**.

---

### *Try post-training quantization*

In [230]:
# use the trained model
converter = tf.lite.TFLiteConverter.from_keras_model(base_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# before convert it , need to add representative data sets to keep a good accuracy    
def representative_dataset():
      for data in x_repre:
        yield [tf.dtypes.cast(data, tf.float32)]
converter.representative_dataset = representative_dataset
tflite_quant_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp_7_zh2__/assets


INFO:tensorflow:Assets written to: /tmp/tmp_7_zh2__/assets


In [231]:
with open('./Quantization/post-train.tflite', 'wb') as f:
    f.write(tflite_quant_model)

In [135]:
len(tflite_quant_model)

2736

In [149]:
def representative_dataset():
    for _ in range(100):
        data = np.random.rand(1, 244, 244, 3)
        yield [data.astype(np.float32)]

In [158]:
x_repre = x_train[:10]
y_repre = y_train[:10]

In [221]:
x_repre.shape

(10, 200, 1)

---

In [125]:
# generate test data

# define parameters
types = 10
basic = 4
rand = np.random.randint(low=0,high=100,size=(types,basic))
sourcelist = []

# create 50 mixed sources
for i in range(types):
    randsource = Simulator.simulator(gt=0.5, Nbins=200, width=1., Ndet=1e6, sps=1., laser=1, ther=0, non=0)
    sourcelist.append(randsource)

# generate 50 data sets per each
x_ = []
y_ = []
datasets = 5
for k, source in enumerate(sourcelist):
    for i in range(datasets): 
        data = source.get_data(dist=source.distribution())
        x_.append(data[0])
        y_.append(data[2])
# convert them to np.array
x_ = np.array(x_)
y_ = np.array(y_)

# shuffle
i = np.random.permutation(len(x_))
x_ = x_[i]
y_ = y_[i]

# reshape
x_train = x_.reshape(x_.shape[0], x_.shape[1], 1)
y_train = y_.reshape(y_.shape[0])
#loss, accuracy = model.evaluate(x_test, y_test)
#print("Test accuracy", accuracy)
#print("Test loss", loss)


/home/laserlab/coral/git/CompPhys/g2-coral/simulation/Simulator.py:196: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  data = np.array([signal, binnumber, np.array([binary])])


In [126]:
# train the base_model
epochs = 10
batch_size = 5

base_model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    loss='mse',
    metrics=['sparse_categorical_accuracy'],
)
history = base_model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    verbose=1,
)


Epoch 1/10
8/8 [==============================] - 0s 10ms/step - loss: 0.5494 - sparse_categorical_accuracy: 0.4301 - val_loss: 0.3000 - val_sparse_categorical_accuracy: 0.7000
Epoch 2/10
8/8 [==============================] - 0s 3ms/step - loss: 0.4476 - sparse_categorical_accuracy: 0.5524 - val_loss: 0.3000 - val_sparse_categorical_accuracy: 0.7000
Epoch 3/10
8/8 [==============================] - 0s 3ms/step - loss: 0.3873 - sparse_categorical_accuracy: 0.6127 - val_loss: 0.3000 - val_sparse_categorical_accuracy: 0.7000
Epoch 4/10
8/8 [==============================] - 0s 2ms/step - loss: 0.4849 - sparse_categorical_accuracy: 0.5151 - val_loss: 0.3000 - val_sparse_categorical_accuracy: 0.7000
Epoch 5/10
8/8 [==============================] - 0s 3ms/step - loss: 0.4447 - sparse_categorical_accuracy: 0.5553 - val_loss: 0.3000 - val_sparse_categorical_accuracy: 0.7000
Epoch 6/10
8/8 [==============================] - 0s 2ms/step - loss: 0.4636 - sparse_categorical_accuracy: 0.5364 - va

In [24]:
# convert the keras model into TF lite
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('G2model.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /tmp/tmp8u51t9oq/assets


In [30]:
def make_model(input_shape):
    # input shape should be (time signal, 1)
    input_layer = keras.layers.Input(input_shape)

    # 1st con1d layer
    conv1 = keras.layers.Conv1D(filters=64, kernel_size=4, padding='valid', strides=1, activation='relu')(input_layer)
    
    # 2nd con1d layer
    conv2 = keras.layers.Conv1D(filters=64, kernel_size=4, padding='valid', strides=1, activation='relu')(conv1)

    # 3rd con1d layer
    conv3 = keras.layers.Conv1D(filters=64, kernel_size=4, padding='valid', strides=1, activation='relu')(conv2)
    
    # maxpooling layer
    pool = keras.layers.MaxPool1D(pool_size=3, strides=1, padding='valid')(conv3) # keras.layers.GlobalAveragePooling1D()(conv3) #  # 
    
    # flatten layer
    flat = keras.layers.Flatten()(pool)
    
    # fully connected layer to output a binary vector
    dense1 = keras.layers.Dense(2, activation='relu')(flat)
    #dense2 = keras.layers.Dense(2, activation='relu')(dense1)

    return keras.models.Model(inputs=input_layer, outputs=dense1)


model = make_model(input_shape=(200,1))


In [35]:
LastValueQuantizer = tfmot.quantization.keras.quantizers.LastValueQuantizer
MovingAverageQuantizer = tfmot.quantization.keras.quantizers.MovingAverageQuantizer

class DefaultDenseQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
    # Configure how to quantize weights.
    def get_weights_and_quantizers(self, layer):
          return [(layer.kernel, LastValueQuantizer(num_bits=8, symmetric=True, narrow_range=False, per_axis=False))]

    # Configure how to quantize activations.
    def get_activations_and_quantizers(self, layer):
          return [(layer.activation, MovingAverageQuantizer(num_bits=8, symmetric=False, narrow_range=False, per_axis=False))]

    def set_quantize_weights(self, layer, quantize_weights):
      # Add this line for each item returned in `get_weights_and_quantizers`
      # , in the same order
          layer.kernel = quantize_weights[0]

    def set_quantize_activations(self, layer, quantize_activations):
      # Add this line for each item returned in `get_activations_and_quantizers`
      # , in the same order.
          layer.activation = quantize_activations[0]

    # Configure how to quantize outputs (may be equivalent to activations).
    def get_output_quantizers(self, layer):
          return []

    def get_config(self):
          return {}


In [43]:
def make_model(input_shape):
    # input shape should be (time signal, 1)
    input_layer = keras.layers.Input(input_shape)

    # 1st con1d layer
    conv1 = quantize_annotate_layer(keras.layers.Conv1D(filters=64, kernel_size=4, padding='valid', strides=1, activation='relu'), DefaultDenseQuantizeConfig())(input_layer)

    # 2nd con1d layer
    conv2 = keras.layers.Conv1D(filters=64, kernel_size=4, padding='valid', strides=1, activation='relu')(conv1)

    # 3rd con1d layer
    conv3 = keras.layers.Conv1D(filters=64, kernel_size=4, padding='valid', strides=1, activation='relu')(conv2)
    
    # maxpooling layer
    pool = keras.layers.MaxPool1D(pool_size=3, strides=1, padding='valid')(conv3) # keras.layers.GlobalAveragePooling1D()(conv3) #  # 
    
    # flatten layer
    flat = keras.layers.Flatten()(pool)
    
    # fully connected layer to output a binary vector
    dense1 = keras.layers.Dense(2, activation='relu')(flat)
    #dense2 = keras.layers.Dense(2, activation='relu')(dense1)

    return keras.models.Model(inputs=input_layer, outputs=dense1)


model = make_model(input_shape=(200,1))

In [48]:
quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_annotate_model = tfmot.quantization.keras.quantize_annotate_model
quantize_scope = tfmot.quantization.keras.quantize_scope

class CustomLayer(tf.keras.layers.Dense):
      pass

model = quantize_annotate_model(tf.keras.Sequential([
   quantize_annotate_layer(CustomLayer(20, input_shape=(20,)), DefaultDenseQuantizeConfig()),
   tf.keras.layers.Flatten()
]))

# `quantize_apply` requires mentioning `DefaultDenseQuantizeConfig` with `quantize_scope`
# as well as the custom Keras layer.
with quantize_scope(
  {'DefaultDenseQuantizeConfig': DefaultDenseQuantizeConfig,
   'CustomLayer': CustomLayer}):
  # Use `quantize_apply` to actually make the model quantization aware.
  quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer_7 (QuantizeLa (None, 20)                3         
_________________________________________________________________
quant_custom_layer_5 (Quanti (None, 20)                425       
_________________________________________________________________
quant_flatten_7 (QuantizeWra (None, 20)                1         
Total params: 429
Trainable params: 420
Non-trainable params: 9
_________________________________________________________________


In [49]:

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.traina


ValueError: Unable to clone model. This generally happens if you used custom Keras layers or objects in your model. Please specify them via `quantize_scope` for your calls to `quantize_model` and `quantize_apply`.